## Appendix: Optimizing data filling via Vectorization 

<div>
<img src="../imgs/vectorization.png" width="500"/>
<figcaption><em>If you're too busy to read the chapter, this is basically it!</em></figcaption>
<div>

### ***PREFACE:***

In our running examples with GDP data, we've typically dealt with datasets that are fairly small by data science standards (~1000 rows or less). 

Modern personal machines and shared computing resources (like Stanford's [Sherlock](https://www.sherlock.stanford.edu/)) are ridiculously powerful, and they can quickly chew through almost any code that you write - even if it has zero optimization.

But what happens if you're dealing with truly massive datasets with hundreds of thousands of rows? With such datasets, code runtime can actually become a constraint on research progression and completion if processing scripts are written suboptimally.

Although the defintion of optimal code and the processing of writing it varies with context, here we're going to learn a technique that can be applied in many situations: *vectorization.*

### ***DISCUSSION 1:***

A natural first question to ask is, *what is vectorizing/vectorization?* You may have heard of vectors before in a mathematics class, where vectors are a series of ordered values like this: [3, 2, 5] or [4, 7, 11, 8]. 

In computer science, vectorization is a technique related to the mathematical definition of a vector - it's the process of converting an algorithm from a scalar implementation (which performs operations on, at most, a pair of operands at once) to a vectorized one (which performs operations on a series of values at once). For a quick example: 

The processing of a scalar implementation of the instruction, "add two to every element in this list," would look like: 

$$input: [3, 2, 5] → [3 + 2 = 5, 2, 5] → [5, 2 + 2 = 4, 5] → [5, 4, 5 + 2 = 7] → output: [5, 4, 7]$$

while the processing of a vectorized implementation of the same instruction would look like: 

$$input: [3, 2, 5] → [3 + 2 = 5, 2 + 2 = 4, 5 + 2 = 7] → output: [5, 4, 7]$$

(where different processors in your machine carry out the '+2' instruction on each element concurrently.)

If the previous example makes sense to you, congratulations! You understand vectorization. Feel free to come up with your own quicker working definition (mine is, "make computer do same thing to many things at same time instead of on things one-by-one." 

### ***DISCUSSION 2:***

In Python, (almost) any looped operation can be vectorized, but why do we need vectorization in the first place? 

<div>
<img src="../imgs/pythonloop.png" width="500"/>
<div>


While loops are a wonderful and flexible programatic idiom, they're also inherently slow due to the **dynamically typed nature of Python.** What does this mean? Let's take a look in the context of executing a program:

When you execute a program in Python, Python:

* First goes line-by-line through your code;

* Compiles it into a machine-readable version of itself called bytecod*; and

* Then this bytecode is executed to actually run the program. 

Suppose your code has a block where you loop over and perform some operation on the elements in a list. As Python is dynamically typed, *it does not know the type of the objects present in the list until it accesses each element.*

In [ ]:
# nothing about the variable name 'a' denotes the type of the contained value. it could be: 
a = 5
print(type(a)) # an int

a = "vectorization rocks!" 
print(type(a)) # a string 

a = dict({"vectorization" : "rocks!"})
print(type(a)) # a dictionary of strings

For any object in Python, the typing information is stored in the the object itself. Therefore, for each iteration in the loop, Python has to perform a series of overhead operations (like determining element type, resolving scope, checking for invalid operations, etc.) until it can carry out the actual operation you instructed it to. As you might imagine, repeatedly performing these overhead operations on massive datasets results in a significantly increased runtime. 

*A quick aside:* A **statically typed** programming language like C avoids this recurring overhead cost by compelling programmers to explicitly denote the type of every object you use. But, these languages come with their own drawbacks. For example, if two external libraries provide functionality for the same concept with differently-typed implementations, library users will have to provide their own translation layer to allow the two libraries to interoperate. In Python, no/few such interoperability issues arise:

In [26]:
# both pandas and numpy implement an array-type collection
import pandas as pd
import numpy as np 

# instantiating a numpy array and a pandas Series: 
a = np.array([1, 2, 3, 4])
b = pd.Series([5, 6, 7, 8])

# we can use both array variables at the same time, with no confusion from Python!
print("The conductor says: " + str(a))
print("And then: " + str(b.values))
print("And the band goes: 🎶🎷🎶! 🎶🎹🎶! 🎶🎻🎶!")

The conductor says: [1 2 3 4]
And then: [5 6 7 8]
And the band goes: 🎶🎷🎶! 🎶🎹🎶! 🎶🎻🎶!


### ***DISCUSSION 3:***

As we continue to run through our list of interrogatives, we've learned the *what* and *why* of vectorization - now it's time to learn the *when!* 

If you recall, we earlier learned that almost any looped operation in Python can be vectorized. To understand the exceptions, we can first classify all looped operations into two types: 

1. Order-dependent operations; and

2. Order-independent operations. 

As the names suggest, the required order (or lack thereof) of the iterations of a looped operation determines its eligibility for vectorization. Consider the following example: 

In [20]:
# suppose we have a Anthony-Land GDP dataset with some discontinuities: 
gdp_data = [(1990, 45), (1991, 46), (1992, 48), (1993, 52), (1995, 55), (1996, 60), (1997, 63), (1999, 67)] # data for the years 1994 and 1998 are missing! 

# if we want to calculate average % increase in gdp from 1990-1999, we have to fill these gaps first! 
#   fill method: gdp_X = avg. of past four years' GDP measurements. 

# 1994 data fill: 
gdp_data.insert(4, (1994, round((gdp_data[0][1] + gdp_data[1][1] + gdp_data[2][1] + gdp_data[3][1])/4))) 
print("After imputing the 1994 data point:" + str(gdp_data))

# 1998 data fill: 
gdp_data.insert(8, (1998, round((gdp_data[4][1] + gdp_data[5][1] + gdp_data[6][1] + gdp_data[7][1])/4)))
print("After imputing the 1998 data point:" + str(gdp_data))

# avg % increase in gdp: 
percent_increase = []
for i in range(len(gdp_data)): 
    if i != 0: percent_increase.append((gdp_data[i][1] - gdp_data[i-1][1])/gdp_data[i][1])

print ("Average % increase in GDP: " + str(round(sum(percent_increase)/len(percent_increase), 4) * 100))

After imputing the 1994 data point:[(1990, 45), (1991, 46), (1992, 48), (1993, 52), (1994, 48), (1995, 55), (1996, 60), (1997, 63), (1999, 67)]
After imputing the 1998 data point:[(1990, 45), (1991, 46), (1992, 48), (1993, 52), (1994, 48), (1995, 55), (1996, 60), (1997, 63), (1998, 56), (1999, 67)]
Average % increase in GDP: 3.94


What if we computed filled the two data points in reverse-order? 

In [21]:
# same data: 
gdp_data = [(1990, 45), (1991, 46), (1992, 48), (1993, 52), (1995, 55), (1996, 60), (1997, 63), (1999, 67)] # data for the years 1994 and 1998 are missing! 

# same fill method: gdp_X = avg. of past four years' GDP measurements. 

# 1998 data fill: 
gdp_data.insert(7, (1998, round((gdp_data[3][1] + gdp_data[4][1] + gdp_data[5][1] + gdp_data[6][1])/4)))
print("After imputing the 1998 data point:" + str(gdp_data))

# 1994 data fill: 
gdp_data.insert(4, (1994, round((gdp_data[0][1] + gdp_data[1][1] + gdp_data[2][1] + gdp_data[3][1])/4))) 
print("After imputing the 1994 data point:" + str(gdp_data))

# avg % increase in gdp: 
percent_increase = []
for i in range(len(gdp_data)): 
    if i != 0: percent_increase.append((gdp_data[i][1] - gdp_data[i-1][1])/gdp_data[i][1])

print ("Average % increase in GDP: " + str(round(sum(percent_increase)/len(percent_increase), 4) * 100))

After imputing the 1998 data point:[(1990, 45), (1991, 46), (1992, 48), (1993, 52), (1995, 55), (1996, 60), (1997, 63), (1998, 58), (1999, 67)]
After imputing the 1994 data point:[(1990, 45), (1991, 46), (1992, 48), (1993, 52), (1994, 48), (1995, 55), (1996, 60), (1997, 63), (1998, 58), (1999, 67)]
Average % increase in GDP: 4.04


A-ha! As you can see, reversing the order that the datapoints were filled in does change the final resulting figure. This is because filling the 1994 data point first makes a 1994 GDP value available as a sample datapoint for imputing the 1998 GDP value, whereas reversing the order of imputation does not. 

As such, the operation we just performed is order-dependent (i.e., the final result is contingent upon the order in which any sub-operations are performed) and ineligible for vectorization. Reason being, *when an operation is vectorized, Python provides no constraints on the order in which the sub-operations are carried out.*

<div>
<img src="../imgs/analogy.png" width="500"/>
<figcaption><em>Vectorized operations are to sets, as scalar looped operations are to lists!</em></figcaption>
<div><br>

Expanding upon our "add 2 to every element in this list" example from before, suppose our input now was much longer: $[3, 2, 5, 4, 6, 11, 14, 8, 1, 37, ... (100,000 \text{ more numbers}) ..., 24, 2].$ Since our computer likely does not have enough processing units to add 2 to every element concurrently, it'll perform the vectorized operation in chunks. We can't guarantee that the first chunk of say, 1000 inputs, coincides with the first 1000 elements in the list. 

But, in cases like these, where the final result will be independent of the order of the sub-operations, this random chunking is totally fine! And for that reason, we say *order-independent* operations can be vectorized. 

*Note:* Determining order-independence (or lack thereof) for any particular operation is a nonstandardized task, but reference to other objects being changed by the same loop is a usually a good sign that order-dependence is present. When in doubt, work with a subset of your data and try switching up the processing order of the loop! Does the result change? 

### ***DISCUSSION 4:***

Now that we've understood the theory, it's time to tackle the practice! 

# **OUTLINE**
* In Python, what tools do we use to vectorize?
  * libraries: pandas (numpy)
  * lineprofiler?
  * timing programs: timeit (+ magic commands)
   
* How do we vectorize? *Coding demonstration* 
  * loop (Pandas, iterrows)
  * apply (applies a function along a specified axis, rows|columns, better than iterrows, but still requires looping through rows, best used when there is no way to vectorize)
  * vectorization in pandas
    * side note: many pandas built-in functions are written as vectorized functions! so they're actually faster if you pass them arrays anyways
  * vectorization with numpy arrays
    * vec in pandas is done on pandas series (which still demands, for a given array, named indexing, data-type checking, etc.)
    * vec in numpy ops is done "under the hood" as optimized, pre-compiled C code on ndarrays 
      * pd .values 
  * what if you really want to loop? 
    * why loop -> function complex, tough to vectorize; vectorize creates memory overhead, stubborn
    * cython (converts/compiles python code as c code)
      * %load_ext cython
      * %%cython (-a command to show how much has been converted to C)
      * cpdef func(): asfkjadjfkdkf
      * run with apply
      * making functions more cython friendly: add typing to function, replace python/numpy libraries with c math libraries
  
* Enjoy making your code run faster? Stay posted for the SSDS MOPS Workshop (**M**anaging & **O**ptimizing **P**erformance and **S**calability - and Memory!)